## Importing Dependencies

In [58]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
chrom_driv="D:\downloads\chromedriver_win32\chromedriver.exe"

In [2]:
url=r"https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
driver = webdriver.Chrome(chrom_driv)
driver.get(url)
get_url=driver.current_url
if get_url==url:
    page_source=driver.page_source

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16964\2210004899.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrom_driv)


In [3]:
soup=BeautifulSoup(page_source,features="html.parser")
type(soup)

bs4.BeautifulSoup

## Part1:

In [4]:
matches=soup.find_all('div',attrs={'class':"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})

In [26]:
match=matches[2]
atag=match.h2.a
product_name=atag.text.strip()
print("NAME : "+product_name)
rating=match.find(class_="a-size-base").text.strip()
print("rating : "+rating)
review=match.find(class_="a-size-base s-underline-text").text
print("No of reviews: ",review)
price=match.find(class_="a-offscreen").text.strip()
print("Price: "+ price)

prod_url= 'https://www.amazon.com'+atag.get('href')
print("product_url: "+prod_url)

NAME : American Tourister 32 Ltrs Black Casual Backpack (AMT FIZZ SCH BAG 02 - BLACK)
rating : 51,380
No of reviews:  51,380
Price: ₹1,199
product_url: https://www.amazon.com/American-Tourister-AMT-SCH-02/dp/B07CJCGM1M/ref=sr_1_3?crid=2M096C61O4MLT&keywords=bags&qid=1675437123&sprefix=ba%2Caps%2C283&sr=8-3


In [28]:
def extract(match):
    atag=match.h2.a
    product_name=atag.text.strip()
    prod_url='https://www.amazon.com'+atag.get('href')
    try:
        rating=match.i.text
        review=match.find(class_="a-size-base s-underline-text").text
    except AttributeError:
        return
    try:
        price=match.find(class_="a-offscreen").text.strip()
    except AttributeError:
        return
    res=(product_name,prod_url,price,rating,review)
    return res
        

    

In [29]:
records=[]
for match in matches:
    record=extract(match)
    if record:
        records.append(record)
    
len(records)

22

In [30]:
records[0]

('URBAN TRIBE Havana 15.6" inch Laptop Backpack for Men and Women | 27L Office/Work/Casual Bag | Water Repellent | Black',
 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozNzUzMzAwODEzODI5ODQ4OjE2NzU0MzcxMjM6c3BfYXRmOjIwMTA0MTQwMjYzNjk4OjowOjo&url=%2FUrban-Tribe-Laptop-Backpack-Havana%2Fdp%2FB01LXNNFDF%2Fref%3Dsr_1_1_sspa%3Fcrid%3D2M096C61O4MLT%26keywords%3Dbags%26qid%3D1675437123%26sprefix%3Dba%252Caps%252C283%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1%26smid%3DA385M0TPSNV7VS',
 '₹1,169',
 '3.9 out of 5 stars',
 '684')

In [10]:
for pg_no in range(1,6):
    base="https://www.amazon.in/s?k=bags&page={}&crid=2M096C61O4MLT&qid=1675415719&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    url=base.format(pg_no)
    driver = webdriver.Chrome(chrom_driv)
    driver.get(url)
    driver.close()

    

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16964\1341365552.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrom_driv)


### Part2:


In [51]:
soup2=BeautifulSoup(page_source1,features="html.parser")
type(soup2)

bs4.BeautifulSoup

In [63]:
#soup.prettify()

In [45]:
def extract_part2(prod):
    driver1 = webdriver.Chrome(chrom_driv)
    #driver.implicitly_wait(2)
    driver1.get(prod_url)
    page_source1=driver1.page_source
    soup2=BeautifulSoup(page_source1,features="html.parser")
    details=soup2.find('div',{'id':"featurebullets_feature_div"}).text
    print(details)
    try: 
        product_descp=soup2.find_all('td',attrs={'class':'apm-top'})
        product_description=""
        for prod in product_descp:
            product_description+=prod.find('h4').text.strip()
    except AttributeError:
        product_description=details
    try:
        
        product_details=soup2.find('ul',attrs={'class':'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list'})
        product=product_details.find_all('span',attrs={'class':'a-list-item'})
        count_real=0
        manufacturer=""
        asin=""
        for i in product:
    #print(i.text.strip())
            if  'Manufacturer' in  i.text.strip():
                if count_real==1:
                    manu=i.text.strip().split(':')[-1]
                    manufacturer=' '.join(manu.split()[1:])
                count_real+=1 
            if 'ASIN' in i.text.strip():
                asin=i.text.strip().split(':')[-1]
                asin=asin.split()[1:][0]
                
    except AttributeError:
        print("details fail")
        return
    
    res=(details,product_description,manufacturer,asin)
    driver1.close()
    return res
    
    
    
    

In [46]:
prod_url=records[3][1]
rec=extract_part2(prod_url)



C:\Users\lenovo\AppData\Local\Temp\ipykernel_16964\3221645102.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(chrom_driv)



  
 This fits your .
     
  Make sure this fits by entering your model number. 
 
 30L Capacity: The Backpack has a padded laptop compartment that can fit 15.6 inch laptop. It can also accommodate essentials such as a Lunch box, Charger, Tablet, Books, Power Bank, Wallet, and Sunglass.    Multi-Level Organizer: The backpack has a front zipper pocket for the ID card, Sanitizer, House keys, Pen. First compartment with pockets for the earphones, pens, notebook, Fitness Band. Spacious second compartment for all your electronics.    Ergonomic Design: Thick cushioning around the shoulders and on the back panel makes the backpack super light and comfortable on the move. “S” Shaped shoulder strap distribute the weight of the backpack across your upper body.    Made In India: Wesley Business Casual Backpack is fully manufactured in India and pass through stringent quality tests and checks    1 Year Warranty: The Wesley Business casual backpack comes with 1 year of warranty    Water Resistant 

In [44]:
rec

In [50]:
data={
        'Product url':[],
        'Prodcut name': [],
        'product Price': [],
        'Rating': [],
        'Number of reviews':[],

            
    }

### Exctacting details of the product up to 20 page

In [51]:
def main():
    base="https://www.amazon.in/s?k=bags&page={}&crid=2M096C61O4MLT&qid=1675415719&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    
    for i in range(1,21):
        records=[]
        driver = webdriver.Chrome(chrom_driv)
        driver.implicitly_wait(8)
        url=base.format(i)
        driver.get(url)
        soup=BeautifulSoup(page_source,features="html.parser")
        matches=soup.find_all('div',attrs={'class':"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})
        for match in matches:
            rec1=extract(match)
            if rec1 :
                data['Prodcut name'].append(rec1[0])
                data['Product url'].append(rec1[1])
                data['product Price'].append(rec1[2])
                data['Rating'].append(rec1[3])
                data['Number of reviews'].append(rec1[4])
main()
             
driver.quit()
df=pd.DataFrame(data)            
                
        

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16964\3894380735.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrom_driv)


In [54]:
df.to_csv('output.csv')

In [57]:
## reading the output file
recs=pd.read_csv('output.csv')
recs.head()

,Unnamed: 0,Product url,Prodcut name,product Price,Rating,Number of reviews
0,0,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,"URBAN TRIBE Havana 15.6"" inch Laptop Backpack ...","₹1,169",3.9 out of 5 stars,684
1,1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,Amazon Basics - Laptop Backpack/Casual Backpac...,"₹1,209",1.0 out of 5 stars,1
2,2,https://www.amazon.com/American-Tourister-AMT-...,American Tourister 32 Ltrs Black Casual Backpa...,"₹1,199",4.1 out of 5 stars,"51,380"
3,3,https://www.amazon.com/Wesley-Milestone-Waterp...,Wesley Milestone 2.0 Casual Waterproof Laptop ...,₹565,4.3 out of 5 stars,"9,233"
4,4,https://www.amazon.com/Skybags-Brat-Black-Casu...,Skybags Brat Black 46 Cms Casual Backpack,₹669,4.1 out of 5 stars,"3,373"
